Figure 1: Illustration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_squared_error

np.random.seed(42)
torch.manual_seed(42)

# NN Definition
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim=2, activation_func='sigmoid'):
        super(NeuralNetwork, self).__init__()
        self.first = nn.Linear(input_dim, 100)
        self.hidden1 = nn.Linear(100, 100)
        self.hidden2 = nn.Linear(100, 100)
        self.hidden3 = nn.Linear(100, 100)
        self.output = nn.Linear(100, 1)
        self.activation = nn.Sigmoid() if activation_func == 'sigmoid' else nn.ReLU()

    def forward(self, x):
        x = self.activation(self.first(x))
        x = self.activation(self.hidden1(x))
        x = self.activation(self.hidden2(x))
        x = self.activation(self.hidden3(x))
        return self.output(x)

# Dataset Generation
def generate_data(n=100, noise_std=0.2, outlier_ratio=0.03):
    x1 = np.linspace(0, 2 * np.pi, int(np.sqrt(n)))
    x2 = np.linspace(0, 2 * np.pi, int(np.sqrt(n)))
    x1_grid, x2_grid = np.meshgrid(x1, x2)
    X = np.vstack((x1_grid.flatten(), x2_grid.flatten())).T
    y_true = np.sin(2 * X[:, 0]) * np.cos(2 * X[:, 1])
    y = y_true + np.random.normal(0, noise_std, X.shape[0])

    # 外れ値の挿入
    n_outliers = int(n * outlier_ratio)
    outlier_indices = np.random.choice(n, n_outliers, replace=False)
    y[outlier_indices] = 5 + np.random.uniform(-0.1, 0.1, n_outliers)

    return X, y, y_true, outlier_indices

def plot_3d_surface(X, y, predict_func, title, outlier_indices, y_true):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(X[:, 0], X[:, 1], y_true, color='blue', label='True Data')
    ax.scatter(X[outlier_indices, 0], X[outlier_indices, 1], y[outlier_indices], color='red', label='Outliers', s=50)

    # Plot prediction surface
    x1_grid, x2_grid = np.meshgrid(np.linspace(0, 2 * np.pi, 100),
                                   np.linspace(0, 2 * np.pi, 100))
    X_grid = np.vstack((x1_grid.flatten(), x2_grid.flatten())).T
    y_grid_pred = predict_func(X_grid).reshape(x1_grid.shape)

    ax.plot_surface(x1_grid, x2_grid, y_grid_pred, cmap='viridis', alpha=0.6)

    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_zlabel('y')
    ax.set_title(title)
    plt.legend()
    plt.show()

# Training and Plot Prediction Models
def train_and_plot(X, y, y_true, outlier_indices, method, weight_decay=0):
    if method == 'Linear+Huber':
        model = HuberRegressor()
        model.fit(X, y)
        predict_func = lambda X_new: model.predict(X_new)
    elif method == 'NN+Huber':
        model = NeuralNetwork()
        optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
        criterion = nn.SmoothL1Loss()
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y.reshape(-1, 1), dtype=torch.float32)
        for epoch in range(5000):
            optimizer.zero_grad()
            outputs = model(X_tensor)
            loss = criterion(outputs, y_tensor)
            loss.backward()
            optimizer.step()
            scheduler.step()
        predict_func = lambda X_new: model(torch.tensor(X_new, dtype=torch.float32)).detach().numpy().flatten()
    elif method == 'NN+Tukey':
        def tukey_loss(output, target, c=4.685):
            residual = target - output
            abs_residual = torch.abs(residual)
            mask = abs_residual <= c
            loss = torch.zeros_like(residual)
            loss[mask] = (c**2 / 6) * (1 - (1 - (residual[mask] / c)**2)**3)
            loss[~mask] = (c**2 / 6)
            return torch.mean(loss)
        model = NeuralNetwork()
        optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y.reshape(-1, 1), dtype=torch.float32)
        for epoch in range(5000):
            optimizer.zero_grad()
            outputs = model(X_tensor)
            loss = tukey_loss(outputs, y_tensor)
            loss.backward()
            optimizer.step()
            scheduler.step()
        predict_func = lambda X_new: model(torch.tensor(X_new, dtype=torch.float32)).detach().numpy().flatten()
    elif method == 'TTL+HOVR(k=2)':
        model = NeuralNetwork()
        xi = nn.Parameter(torch.zeros(X.shape[0], 1), requires_grad=True)
        optimizer = optim.Adam(list(model.parameters()) + [xi], lr=0.01, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y.reshape(-1, 1), dtype=torch.float32)
        h = int(0.9 * X.shape[0])
        lambd = 1e-3
        k = 2
        q = 2
        for epoch in range(5000):
            optimizer.zero_grad()
            preds = model(X_tensor)
            residuals = (y_tensor - preds).view(-1, 1)
            loss_fit = (1 / X.shape[0]) * torch.sum((residuals - xi) ** 2)
            xi_squared = xi.view(-1) ** 2
            T_h_xi = (1 / X.shape[0]) * torch.sum(torch.topk(xi_squared, h, largest=False)[0])
            hovr_term = lambd * hovr_regularization(model, X_tensor, k, q)
            total_loss = loss_fit + T_h_xi + hovr_term
            total_loss.backward()
            optimizer.step()
            scheduler.step()
        predict_func = lambda X_new: model(torch.tensor(X_new, dtype=torch.float32)).detach().numpy().flatten()

    plot_3d_surface(X, y, predict_func, method, outlier_indices, y_true)



# HOVR
def hovr_regularization(model, x, k=2, q=2, M=10):
    x_min, x_max = x.min(0)[0], x.max(0)[0]
    random_points = torch.tensor(np.random.uniform(x_min.numpy(), x_max.numpy(), (M, x.shape[1])), dtype=torch.float32, requires_grad=True)
    preds = model(random_points)
    grads = torch.autograd.grad(preds, random_points, torch.ones_like(preds), create_graph=True)[0]
    hovr_term = 0.0
    n_dims = x.shape[1]
    for i in range(n_dims):
        grad_i = grads[:, i]
        temp_grad = grad_i
        for _ in range(k - 1):
            temp_grad = torch.autograd.grad(temp_grad, random_points, torch.ones_like(temp_grad), create_graph=True)[0][:, i]
        hovr_term += (1 / n_dims) * torch.sum(torch.abs(temp_grad) ** q)
    return hovr_term

# メインスクリプト
X, y, y_true, outlier_indices = generate_data()

methods = ['Linear+Huber', 'NN+Huber', 'NN+Tukey', 'TTL+HOVR(k=2)']
for method in methods:
    train_and_plot(X, y, y_true, outlier_indices, method)